In [1]:
# load the json file test_queries.json
import json

test_queries = json.load(open('test_queries.json'))

In [3]:
'''
test_queries looks like this:

[
    {
      "task": "renewable_energy_presentation",
      "prompt": "Create a presentation about the benefits of renewable energy. Include a pie chart showing the global energy mix and a bar plot comparing the costs of different energy sources. Use a green technology theme and make it look professional."
    },
    {
      "task": "ai_healthcare_research_summary",
      "prompt": "Make a presentation summarizing the key findings of the research paper 'Advances in AI for Healthcare' by John Doe et al. Include the most important figures and tables from the paper. Use a clean, minimalistic theme suitable for an academic audience."
    },
    ...
'''
# print queries whose task is 'renewable_energy_presentation', 'product_sales_data_formatting', 'software_engineer_cover_letter'
filtered_queries = [query for query in test_queries if query['task'] in ['renewable_energy_presentation', 'product_sales_data_formatting', 'software_engineer_cover_letter']]
print(json.dumps(filtered_queries, indent=2))

[
  {
    "task": "renewable_energy_presentation",
    "prompt": "Create a presentation about the benefits of renewable energy. Include a pie chart showing the global energy mix and a bar plot comparing the costs of different energy sources. Use a green technology theme and make it look professional."
  },
  {
    "task": "product_sales_data_formatting",
    "prompt": "Create an Excel sheet with a table of sales data for different products. Include columns for product name, unit price, quantity sold, and total revenue. Calculate the average price and total revenue for each product. Format the column names in bold and merge the cells in the header row."
  },
  {
    "task": "software_engineer_cover_letter",
    "prompt": "Draft a professional cover letter for a software engineer position at Google. Highlight the candidate's experience with Python, machine learning, and web development. Use a formal tone and a clean, modern template."
  }
]


In [4]:
'''
Rough notes:
UFO paper - https://arxiv.org/abs/2402.07939
OS Copilot paper - https://arxiv.org/pdf/2402.07456
ScreenAgent paper - https://arxiv.org/abs/2402.07945
'''

'\nRough notes:\nUFO paper - https://arxiv.org/abs/2402.07939\n'

In [39]:
import os
import re

def prompt_loader(agent_name):
    file_name = f"../api/prompt_lib/{agent_name}.txt"
    
    if not os.path.exists(file_name):
        return None
    
    with open(file_name, 'r') as file:
        content = file.read()
    
    prompt_template = {
        'system_prompt': '',
        'fewshot_messages': []
    }
    
    system_prompt_pattern = r'\[\[prompt_template-system_prompt\]\](.*?)\[\[prompt_template-system_prompt\]\]'
    system_prompt_match = re.search(system_prompt_pattern, content, re.DOTALL)
    
    if system_prompt_match:
        prompt_template['system_prompt'] = system_prompt_match.group(1).replace('\\"', '"').strip()
    
    fewshot_pattern = r'\[\[prompt_template-fewshot\]\](.*?)\[\[prompt_template-fewshot\]\]'
    fewshot_match = re.search(fewshot_pattern, content, re.DOTALL)
    
    if fewshot_match:
        fewshot_content = fewshot_match.group(1).replace('\\"', '"').strip()
        fewshot_messages = re.split(r'\[\[prompt_template-fewshot-(?:user|assistant)\]\]', fewshot_content)
        
        for message in fewshot_messages:
            message = message.strip()
            if message:
                if message.startswith('{') and message.endswith('}'):
                    prompt_template['fewshot_messages'].append({'role': 'assistant', 'content': message})
                else:
                    prompt_template['fewshot_messages'].append({'role': 'user', 'content': message})
    
    return prompt_template
task_corrector_prompts = prompt_loader('task_corrector')

In [40]:
prompt_templates = prompt_loader("task_corrector")
prompt_templates['fewshot_messages']

[{'role': 'user',
  'content': 'Correct the following task\n<task>\nWrite a press release announcing the launch of a new sustainable fashion line. Include quotes from the designer and information about the eco-friendly materials used. Format the document with a header, subheadings, and a footer with contact information.\n</task>\n\nNew App list:\n<app_list>\n{\n    "AllUsersPrograms": {\n      "Access.lnk": null,\n      "Brave.lnk": null,\n    },\n    "StartMenu": {\n      "Programs": {\n        "Administrative Tools.lnk": null,\n        "F1_23.lnk": null,\n        "File Explorer.lnk": null\n      }\n    },\n    "AllUsersDesktop": {\n      "Brave.lnk": null,\n      "VLC media player.lnk": null\n    }\n  }\n</app_list>'},
 {'role': 'assistant',
  'content': '{"corrected_task": "Write a press release using Google Docs announcing the launch of a new sustainable fashion line. Use Brave to google information about a new sustainable fashion line and include quotes from the designer and infor

In [13]:
import re

def xmlResponseToDict(xml_response):
    def parse_tag(tag_content):
        if not tag_content:
            return None
        
        tag_pattern = re.compile(r'<(\w+)>(.*?)</\1>', re.DOTALL)
        tag_match = tag_pattern.match(tag_content)
        
        if tag_match:
            tag_name = tag_match.group(1)
            tag_value = tag_match.group(2).strip()
            
            if tag_name.endswith('_list'):
                items = re.findall(r'<(\w+)>(.*?)</\1>', tag_value, re.DOTALL)
                
                if items:
                    parsed_items = []
                    for item_name, item_content in items:
                        parsed_item = parse_tag(f'<{item_name}>{item_content}</{item_name}>')
                        if parsed_item:
                            parsed_items.append(parsed_item)
                    
                    return {tag_name: parsed_items}
                else:
                    items = re.split(r'\s*\n\s*', tag_value)
                    items = [item.strip() for item in items if item.strip()]
                    return {tag_name: items}
            else:
                # Recursively parse nested tags
                nested_tags = re.findall(r'<(\w+)>(.*?)</\1>', tag_value, re.DOTALL)
                if nested_tags:
                    parsed_nested_tags = {}
                    for nested_tag_name, nested_tag_content in nested_tags:
                        parsed_nested_tag = parse_tag(f'<{nested_tag_name}>{nested_tag_content}</{nested_tag_name}>')
                        if parsed_nested_tag:
                            parsed_nested_tags.update(parsed_nested_tag)
                    return {tag_name: parsed_nested_tags}
                else:
                    return {tag_name: tag_value}
        
        return None

    tags = re.findall(r'<(\w+)>(.*?)</\1>', xml_response, re.DOTALL)
    result = {}
    
    for tag_name, tag_content in tags:
        parsed_tag = parse_tag(f'<{tag_name}>{tag_content}</{tag_name}>')
        if parsed_tag:
            result.update(parsed_tag)
    
    return result

In [14]:
xml_test_string = '''
<step_list>
    <step_1>
        <action_desc>
            Enter info in the first cell
        </action_desc>
        <action_type>
            PRESS
        </action_type>
        <parameters>
            <text_list>
                name
            </text_list>
        </parameters>
    </step_1>

    <step_2>
        <action_desc>
            Move to the right column
        </action_desc>
        <action_type>
            PRESS
        </action_type>
        <parameters>
            <key_list>
                right
            </key_list>
        </parameters>
    </step_2>

    <step_3>
        <action_desc>
            Enter some info in the new cell
        </action_desc>
        <action_type>
            PRESS
        </action_type>
        <parameters>
            <text_list>
                address
            </text_list>
        </parameters>
    </step_3>

    <step_4>
        <action_desc>
            Navigate to the cell under the name field
        </action_desc>
        <action_type>
            PRESS
        </action_type>
        <parameters>
            <key_list>
                down
                left
                left
            </key_list>
        </parameters>
    </step_4>

    <step_5>
        <action_desc>
            Type in some info
        </action_desc>
        <action_type>
            PRESS
        </action_type>
        <parameters>
            <text_list>
                Mango
            </text_list>
        </parameters>
    </step_5>

    <step_6>
        <action_desc>
            Navigate to the cell under the address field
        </action_desc>
        <action_type>
            PRESS
        </action_type>
        <parameters>
            <key_list>
                right
            </key_list>
        </parameters>
    </step_6>

    <step_7>
        <action_desc>
            Type in some info
        </action_desc>
        <action_type>
            PRESS
        </action_type>
        <parameters>
            <text_list>
                Mango Market Street
            </text_list>
        </parameters>
    </step_7>
</step_list>
'''

In [15]:
dict_test = xmlResponseToDict(xml_test_string)
# pretty print the dict
import json
print(json.dumps(dict_test, indent=2))

{
  "step_list": [
    {
      "step_1": {
        "action_desc": "Enter info in the first cell",
        "action_type": "PRESS",
        "parameters": {
          "text_list": [
            "name"
          ]
        }
      }
    },
    {
      "step_2": {
        "action_desc": "Move to the right column",
        "action_type": "PRESS",
        "parameters": {
          "key_list": [
            "right"
          ]
        }
      }
    },
    {
      "step_3": {
        "action_desc": "Enter some info in the new cell",
        "action_type": "PRESS",
        "parameters": {
          "text_list": [
            "address"
          ]
        }
      }
    },
    {
      "step_4": {
        "action_desc": "Navigate to the cell under the name field",
        "action_type": "PRESS",
        "parameters": {
          "key_list": [
            "down",
            "left",
            "left"
          ]
        }
      }
    },
    {
      "step_5": {
        "action_desc": "Type in some in

In [22]:
import re

def xmlResponseToDict(xml_response):
    def parse_tag(tag_content):
        tag_pattern = re.compile(r'<(\w+)>(.*?)</\1>', re.DOTALL)
        tag_match = tag_pattern.match(tag_content)
        
        if tag_match:
            tag_name, tag_value = tag_match.groups()
            
            if tag_name.endswith('_list'):
                items = re.findall(r'<(\w+)>(.*?)</\1>', tag_value, re.DOTALL)
                if items:
                    return [{item[0]: parse_tag(item[1])} for item in items]
                else:
                    return re.split(r'\n', tag_value.strip())
            
            return tag_value.strip()
        
        return None

    tags = re.findall(r'<(\w+)>(.*?)</\1>', xml_response, re.DOTALL)
    result = {}
    for tag in tags:
        parsed_tag = parse_tag(tag[1])
        if parsed_tag is not None:
            result[tag[0]] = parsed_tag
    return result

In [5]:
test = """
this is a test of string formating:
{{FORMAT_ME}}
"""

test.format(FORMAT_ME="Hello World")

'\nthis is a test of string formating:\n{FORMAT_ME}\n'

In [5]:

frequently_used_hotkey_function_desc = {
    "close_workbook": {
        "description": "Close a workbook.",
        "hotkey": "Ctrl+W"
    },
    "open_workbook": {
        "description": "Open a workbook.",
        "hotkey": "Ctrl+O"
    },
    "go_to_home_tab": {
        "description": "Go to the Home tab.",
        "hotkey": "Alt+H"
    },
    "save_workbook": {
        "description": "Save a workbook.",
        "hotkey": "Ctrl+S"
    },
    "copy_selection": {
        "description": "Copy selection.",
        "hotkey": "Ctrl+C"
    },
    "paste_selection": {
        "description": "Paste selection.",
        "hotkey": "Ctrl+V"
    },
    "undo_recent_action": {
        "description": "Undo recent action.",
        "hotkey": "Ctrl+Z"
    },
    "remove_cell_contents": {
        "description": "Remove cell contents.",
        "hotkey": "Delete"
    },
    "choose_fill_color": {
        "description": "Choose a fill color.",
        "hotkey": "Alt+H, H"
    },
    "cut_selection": {
        "description": "Cut selection.",
        "hotkey": "Ctrl+X"
    },
    "go_to_insert_tab": {
        "description": "Go to the Insert tab.",
        "hotkey": "Alt+N"
    },
    "apply_bold_formatting": {
        "description": "Apply bold formatting.",
        "hotkey": "Ctrl+B"
    },
    "center_align_cell_contents": {
        "description": "Center align cell contents.",
        "hotkey": "Alt+H, A, C"
    },
    "go_to_page_layout_tab": {
        "description": "Go to the Page Layout tab.",
        "hotkey": "Alt+P"
    },
    "go_to_data_tab": {
        "description": "Go to the Data tab.",
        "hotkey": "Alt+A"
    },
    "go_to_view_tab": {
        "description": "Go to the View tab.",
        "hotkey": "Alt+W"
    },
    "open_context_menu": {
        "description": "Open the context menu.",
        "hotkey": "Shift+F10"
    },
    "add_borders": {
        "description": "Add borders.",
        "hotkey": "Alt+H, B"
    },
    "delete_column": {
        "description": "Delete column.",
        "hotkey": "Alt+H, D, C"
    },
    "go_to_formula_tab": {
        "description": "Go to the Formula tab.",
        "hotkey": "Alt+M"
    },
    "hide_selected_rows": {
        "description": "Hide the selected rows.",
        "hotkey": "Ctrl+9"
    },
    "hide_selected_columns": {
        "description": "Hide the selected columns.",
        "hotkey": "Ctrl+0"
    }
}

ribbon_keyboard_hotkey_function_desc = {
    "open_home_tab": {
        "description": "Open the Home tab.",
        "hotkey": "Alt+H"
    },
    "move_to_search_field": {
        "description": "Move to the Tell me or Search field.",
        "hotkey": "Alt+Q"
    },
    "open_file_menu": {
        "description": "Open the File menu.",
        "hotkey": "Alt+F"
    },
    "open_insert_tab": {
        "description": "Open the Insert tab.",
        "hotkey": "Alt+N"
    },
    "open_page_layout_tab": {
        "description": "Open the Page Layout tab.",
        "hotkey": "Alt+P"
    },
    "open_formulas_tab": {
        "description": "Open the Formulas tab.",
        "hotkey": "Alt+M"
    },
    "open_data_tab": {
        "description": "Open the Data tab.",
        "hotkey": "Alt+A"
    },
    "open_review_tab": {
        "description": "Open the Review tab.",
        "hotkey": "Alt+R"
    },
    "open_view_tab": {
        "description": "Open the View tab.",
        "hotkey": "Alt+W"
    }
}

navigation_hotkey_function_desc = {
    "move_to_previous_cell": {
        "description": "Move to the previous cell in a worksheet.",
        "hotkey": "Shift+Tab"
    },
    "move_one_cell_up": {
        "description": "Move one cell up in a worksheet.",
        "hotkey": "Up arrow key"
    },
    "move_one_cell_down": {
        "description": "Move one cell down in a worksheet.",
        "hotkey": "Down arrow key"
    },
    "move_one_cell_left": {
        "description": "Move one cell left in a worksheet.",
        "hotkey": "Left arrow key"
    },
    "move_one_cell_right": {
        "description": "Move one cell right in a worksheet.",
        "hotkey": "Right arrow key"
    },
    "move_to_edge_of_current_data_region": {
        "description": "Move to the edge of the current data region.",
        "hotkey": "Ctrl+Arrow key"
    },
    "enter_end_mode": {
        "description": "Enter the End mode, move to the next nonblank cell.",
        "hotkey": "End, Arrow key"
    },
    "move_to_last_cell_on_worksheet": {
        "description": "Move to the last cell on a worksheet.",
        "hotkey": "Ctrl+End"
    },
    "extend_selection_to_last_used_cell": {
        "description": "Extend the selection of cells to the last used cell.",
        "hotkey": "Ctrl+Shift+End"
    },
    "move_to_upper_left_corner_with_scroll_lock": {
        "description": "Move to the cell in the upper-left corner with Scroll lock.",
        "hotkey": "Home+Scroll lock"
    },
    "move_to_beginning_of_worksheet": {
        "description": "Move to the beginning of a worksheet.",
        "hotkey": "Ctrl+Home"
    },
    "move_one_screen_down": {
        "description": "Move one screen down in a worksheet.",
        "hotkey": "Page down"
    },
    "move_to_next_sheet_in_workbook": {
        "description": "Move to the next sheet in a workbook.",
        "hotkey": "Ctrl+Page down"
    },
    "move_one_screen_to_right": {
        "description": "Move one screen to the right in a worksheet.",
        "hotkey": "Alt+Page down"
    },
    "move_one_screen_up": {
        "description": "Move one screen up in a worksheet.",
        "hotkey": "Page up"
    },
    "move_one_screen_to_left": {
        "description": "Move one screen to the left in a worksheet.",
        "hotkey": "Alt+Page up"
    },
    "move_to_previous_sheet_in_workbook": {
        "description": "Move to the previous sheet in a workbook.",
        "hotkey": "Ctrl+Page up"
    }
}

formatting_hotkey_function_desc = {
    "open_format_cells_dialog_box": {
        "description": "Open the Format Cells dialog box.",
        "hotkey": "Ctrl+1"
    },
    "format_fonts": {
        "description": "Format fonts in the Format Cells dialog box.",
        "hotkey": "Ctrl+Shift+F"
    },
    "edit_active_cell": {
        "description": "Edit the active cell.",
        "hotkey": "F2"
    },
    "insert_note": {
        "description": "Insert a note.",
        "hotkey": "Shift+F2"
    },
    "open_insert_dialog_box": {
        "description": "Open the Insert dialog box.",
        "hotkey": "Ctrl+Shift+Plus sign (+)"
    },
    "open_delete_dialog_box": {
        "description": "Open the Delete dialog box.",
        "hotkey": "Ctrl+Minus sign (-)"
    },
    "enter_current_time": {
        "description": "Enter the current time.",
        "hotkey": "Ctrl+Shift+Colon (:)"
    },
    "enter_current_date": {
        "description": "Enter the current date.",
        "hotkey": "Ctrl+Semicolon (;)"
    },
    "switch_display_cell_values_or_formulas": {
        "description": "Switch between displaying cell values or formulas.",
        "hotkey": "Ctrl+Grave accent (`)"
    },
    "copy_formula_from_cell_above": {
        "description": "Copy a formula from the cell above.",
        "hotkey": "Ctrl+Apostrophe (')"
    },
    "move_selected_cells": {
        "description": "Move the selected cells.",
        "hotkey": "Ctrl+X"
    },
    "copy_selected_cells": {
        "description": "Copy the selected cells.",
        "hotkey": "Ctrl+C"
    },
    "paste_content_at_insertion_point": {
        "description": "Paste content at the insertion point.",
        "hotkey": "Ctrl+V"
    },
    "open_paste_special_dialog_box": {
        "description": "Open the Paste Special dialog box.",
        "hotkey": "Ctrl+Alt+V"
    },
    "italicize_text_or_remove_italic_formatting": {
        "description": "Italicize text or remove italic formatting.",
        "hotkey": "Ctrl+I"
    },
    "bold_text_or_remove_bold_formatting": {
        "description": "Bold text or remove bold formatting.",
        "hotkey": "Ctrl+B"
    },
    "underline_text_or_remove_underline": {
        "description": "Underline text or remove underline.",
        "hotkey": "Ctrl+U"
    },
    "apply_or_remove_strikethrough_formatting": {
        "description": "Apply or remove strikethrough formatting.",
        "hotkey": "Ctrl+5"
    },
    "switch_between_hiding_objects_and_displaying_placeholders": {
        "description": "Switch between hiding objects and displaying placeholders.",
        "hotkey": "Ctrl+6"
    },
    "apply_outline_border": {
        "description": "Apply an outline border.",
        "hotkey": "Ctrl+Shift+Ampersand sign (&)"
    },
    "remove_outline_border": {
        "description": "Remove the outline border.",
        "hotkey": "Ctrl+Shift+Underscore (_)"
    },
    "display_or_hide_outline_symbols": {
        "description": "Display or hide the outline symbols.",
        "hotkey": "Ctrl+8"
    },
    "use_fill_down_command": {
        "description": "Use the Fill Down command.",
        "hotkey": "Ctrl+D"
    },
    "apply_general_number_format": {
        "description": "Apply the General number format.",
        "hotkey": "Ctrl+Shift+Tilde sign (~)"
    },
    "apply_currency_format": {
        "description": "Apply the Currency format.",
        "hotkey": "Ctrl+Shift+Dollar sign ($)"
    },
    "apply_percentage_format": {
        "description": "Apply the Percentage format.",
        "hotkey": "Ctrl+Shift+Percent sign (%)"
    },
    "apply_scientific_number_format": {
        "description": "Apply the Scientific number format.",
        "hotkey": "Ctrl+Shift+Caret sign (^)"
    },
    "apply_date_format": {
        "description": "Apply the Date format.",
        "hotkey": "Ctrl+Shift+Number sign (#)"
    },
    "apply_time_format": {
        "description": "Apply the Time format.",
        "hotkey": "Ctrl+Shift+At sign (@)"
    },
    "apply_number_format": {
        "description": "Apply the Number format.",
        "hotkey": "Ctrl+Shift+Exclamation point (!)"
    },
    "open_insert_hyperlink_dialog_box": {
        "description": "Open the Insert hyperlink dialog box.",
        "hotkey": "Ctrl+K"
    },
    "check_spelling": {
        "description": "Check spelling.",
        "hotkey": "F7"
    },
    "display_quick_analysis_options": {
        "description": "Display the Quick Analysis options.",
        "hotkey": "Ctrl+Q"
    },
    "display_create_table_dialog_box": {
        "description": "Display the Create Table dialog box.",
        "hotkey": "Ctrl+L"
    },
    "open_workbook_statistics_dialog_box": {
        "description": "Open the Workbook Statistics dialog box.",
        "hotkey": "Ctrl+Shift+G"
    }
}

selections_and_actions_hotkey_function_desc = {
    "select_entire_worksheet": {
        "description": "Select the entire worksheet.",
        "hotkey": "Ctrl+A"
    },
    "select_current_and_next_sheet_in_workbook": {
        "description": "Select the current and next sheet in a workbook.",
        "hotkey": "Ctrl+Shift+Page down"
    },
    "select_current_and_previous_sheet_in_workbook": {
        "description": "Select the current and previous sheet in a workbook.",
        "hotkey": "Ctrl+Shift+Page up"
    },
    "extend_selection_of_cells_by_one_cell": {
        "description": "Extend the selection of cells by one cell.",
        "hotkey": "Shift+Arrow key"
    },
    "extend_selection_of_cells_to_last_nonblank_cell": {
        "description": "Extend the selection of cells to the last nonblank cell.",
        "hotkey": "Ctrl+Shift+Arrow key"
    },
    "turn_extend_mode_on": {
        "description": "Turn extend mode on and use the arrow keys to extend a selection.",
        "hotkey": "F8"
    },
    "add_non_adjacent_cell_or_range_to_selection": {
        "description": "Add a non-adjacent cell or range to a selection.",
        "hotkey": "Shift+F8"
    },
    "start_new_line_in_same_cell": {
        "description": "Start a new line in the same cell.",
        "hotkey": "Alt+Enter"
    },
    "fill_selected_cell_range_with_current_entry": {
        "description": "Fill the selected cell range with the current entry.",
        "hotkey": "Ctrl+Enter"
    },
    "complete_cell_entry_and_select_cell_above": {
        "description": "Complete a cell entry and select the cell above.",
        "hotkey": "Shift+Enter"
    },
    "select_entire_column": {
        "description": "Select an entire column.",
        "hotkey": "Ctrl+Spacebar"
    },
    "select_entire_row": {
        "description": "Select an entire row.",
        "hotkey": "Shift+Spacebar"
    },
    "select_all_objects_on_worksheet": {
        "description": "Select all objects on a worksheet.",
        "hotkey": "Ctrl+Shift+Spacebar"
    },
    "extend_selection_of_cells_to_beginning_of_worksheet": {
        "description": "Extend the selection of cells to the beginning of the worksheet.",
        "hotkey": "Ctrl+Shift+Home"
    },
    "select_current_region_around_active_cell": {
        "description": "Select the current region around the active cell.",
        "hotkey": "Ctrl+Shift+Asterisk sign (*)"
    },
    "display_first_command_on_menu": {
        "description": "Select the first command on the menu.",
        "hotkey": "Home"
    },
    "repeat_last_command_or_action": {
        "description": "Repeat the last command or action.",
        "hotkey": "Ctrl+Y"
    },
    "undo_last_action": {
        "description": "Undo the last action.",
        "hotkey": "Ctrl+Z"
    },
    "expand_grouped_rows_or_columns": {
        "description": "Expand grouped rows or columns.",
        "hotkey": "Shift+Scroll down"
    },
    "collapse_grouped_rows_or_columns": {
        "description": "Collapse grouped rows or columns.",
        "hotkey": "Shift+Scroll up"
    }
}

data_functions_hotkey_function_desc = {
    "turn_on_or_off_tooltips_for_checking_formulas": {
        "description": "Turn on or off tooltips for checking formulas.",
        "hotkey": "Ctrl+Alt+P"
    },
    "expand_or_collapse_formula_bar": {
        "description": "Expand or collapse the formula bar.",
        "hotkey": "Ctrl+Shift+U"
    },
    "cancel_entry_in_cell_or_formula_bar": {
        "description": "Cancel an entry in the cell or formula bar.",
        "hotkey": "Esc"
    },
    "complete_entry_in_formula_bar_and_select_cell_below": {
        "description": "Complete an entry in the formula bar and select the cell below.",
        "hotkey": "Enter"
    },
    "move_cursor_to_end_of_text_in_formula_bar": {
        "description": "Move the cursor to the end of the text in the formula bar.",
        "hotkey": "Ctrl+End"
    },
    "select_all_text_in_formula_bar_from_cursor_to_end": {
        "description": "Select all text in the formula bar from the cursor to the end.",
        "hotkey": "Ctrl+Shift+End"
    },
    "calculate_all_worksheets": {
        "description": "Calculate all worksheets.",
        "hotkey": "F9"
    },
    "calculate_active_worksheet": {
        "description": "Calculate the active worksheet.",
        "hotkey": "Shift+F9"
    },
    "calculate_all_worksheets_regardless_of_changes": {
        "description": "Calculate all worksheets regardless of changes.",
        "hotkey": "Ctrl+Alt+F9"
    },
    "check_dependent_formulas_and_calculate_all_cells": {
        "description": "Check dependent formulas and calculate all cells.",
        "hotkey": "Ctrl+Alt+Shift+F9"
    },
    "display_menu_or_message_for_error_checking_button": {
        "description": "Display the menu or message for an Error Checking button.",
        "hotkey": "Alt+Shift+F10"
    },
    "display_function_arguments_dialog_box": {
        "description": "Display the Function Arguments dialog box.",
        "hotkey": "Ctrl+A"
    },
    "insert_argument_names_and_parentheses_in_formula": {
        "description": "Insert argument names and parentheses in a formula.",
        "hotkey": "Ctrl+Shift+A"
    },
    "insert_auto_sum_formula": {
        "description": "Insert the AutoSum formula.",
        "hotkey": "Alt+Equal sign (=)"
    },
    "invoke_flash_fill": {
        "description": "Invoke Flash Fill.",
        "hotkey": "Ctrl+E"
    },
    "cycle_through_all_combinations_of_absolute_and_relative_references": {
        "description": "Cycle through all combinations of absolute and relative references.",
        "hotkey": "F4"
    },
    "insert_function": {
        "description": "Insert a function.",
        "hotkey": "Shift+F3"
    },
    "copy_value_from_cell_above": {
        "description": "Copy the value from the cell above.",
        "hotkey": "Ctrl+Shift+Straight quotation mark (\")"
    },
    "create_embedded_chart": {
        "description": "Create an embedded chart.",
        "hotkey": "Alt+F1"
    },
    "create_chart_in_separate_chart_sheet": {
        "description": "Create a chart in a separate Chart sheet.",
        "hotkey": "F11"
    },
    "define_name_to_use_in_references": {
        "description": "Define a name to use in references.",
        "hotkey": "Alt+M, M, D"
    },
    "paste_name_from_paste_name_dialog_box": {
        "description": "Paste a name from the Paste Name dialog box.",
        "hotkey": "F3"
    },
    "calculate_array_formula": {
        "description": "Calculate an array formula.",
        "hotkey": "Enter"
    }
}

objects_hotkey_function_desc = {
    "select_object_under_other_objects": {
        "description": "Select an object that is under other objects.",
        "hotkey": "Ctrl+click"
    },
    "select_previous_object": {
        "description": "Select the previous object.",
        "hotkey": "Shift+Tab"
    },
    "select_next_object": {
        "description": "Select the next object.",
        "hotkey": "Tab key"
    },
    "open_group_dialog_box": {
        "description": "Open the Group dialog box.",
        "hotkey": "Ctrl+Shift+G"
    },
    "open_hyperlink_dialog_box": {
        "description": "Open the Hyperlink dialog box.",
        "hotkey": "Ctrl+Shift+H"
    },
    "open_justify_dialog_box": {
        "description": "Open the Justify dialog box.",
        "hotkey": "Ctrl+Shift+J"
    },
    "toggle_autofilter_on_and_off": {
        "description": "Toggle Autofilter on and off.",
        "hotkey": "Ctrl+Shift+L"
    },
    "open_insert_function_dialog_box": {
        "description": "Open the Insert Function dialog box.",
        "hotkey": "Ctrl+Shift+M"
    },
    "open_name_manager_dialog_box": {
        "description": "Open the Name Manager dialog box.",
        "hotkey": "Ctrl+Shift+N"
    },
    "select_all_cells_with_comments": {
        "description": "Select all cells with comments.",
        "hotkey": "Ctrl+Shift+O"
    },
    "open_print_dialog_box": {
        "description": "Open the Print dialog box.",
        "hotkey": "Ctrl+Shift+P"
    },
    "open_sort_dialog_box": {
        "description": "Open the Sort dialog box.",
        "hotkey": "Ctrl+Shift+Q"
    },
    "open_remove_duplicates_dialog_box": {
        "description": "Open the Remove Duplicates dialog box.",
        "hotkey": "Ctrl+Shift+R"
    },
    "open_styles_dialog_box": {
        "description": "Open the Styles dialog box.",
        "hotkey": "Ctrl+Shift+S"
    },
    "open_themes_dialog_box": {
        "description": "Open the Themes dialog box.",
        "hotkey": "Ctrl+Shift+T"
    },
    "expand_or_collapse_formula_bar": {
        "description": "Expand or collapse the formula bar.",
        "hotkey": "Ctrl+Shift+U"
    },
    "open_paste_special_dialog_box": {
        "description": "Open the Paste Special dialog box.",
        "hotkey": "Ctrl+Shift+V"
    },
    "open_workbook_statistics_dialog_box": {
        "description": "Open the Workbook Statistics dialog box.",
        "hotkey": "Ctrl+Shift+W"
    },
    "repeat_last_action": {
        "description": "Repeat the last action.",
        "hotkey": "Ctrl+Shift+Y"
    },
    "undo_last_action": {
        "description": "Undo the last action.",
        "hotkey": "Ctrl+Shift+Z"
    },
    "check_dependent_formulas_and_calculate_all_cells": {
        "description": "Check dependent formulas and calculate all cells.",
        "hotkey": "Alt+Shift+Ctrl+F9"
    }
}

# hotkey_function_desc = frequently_used_hotkey_function_desc + ribbon_keyboard_hotkey_function_desc + navigation_hotkey_function_desc + formatting_hotkey_function_desc + selections_and_actions_hotkey_function_desc + data_functions_hotkey_function_desc + objects_hotkey_function_desc
hotkey_function_desc = {**frequently_used_hotkey_function_desc, **ribbon_keyboard_hotkey_function_desc, **navigation_hotkey_function_desc, **formatting_hotkey_function_desc, **selections_and_actions_hotkey_function_desc, **data_functions_hotkey_function_desc, **objects_hotkey_function_desc}

In [10]:
# count how many items in hotkey_function_desc
print(len(navigation_hotkey_function_desc))

17


In [1]:

import pyautogui

def move_to_cell(cell_address):
    """
    Moves the cursor to a specific cell in Excel.

    Args:
        cell_address (str): The address of the cell (e.g., 'A1', 'B3').
    """
    pyautogui.press('esc')  # Ensure we're in normal mode
    pyautogui.hotkey('ctrl', 'g')  # Open the "Go To" dialog
    pyautogui.write(cell_address)  # Enter the cell address
    pyautogui.press('enter')  # Confirm the cell address

def input_data_in_cell(data):
    """
    Inputs data into the currently active cell in Excel.

    Args:
        data (str): The data to be entered into the cell.
    """
    pyautogui.write(data)
    pyautogui.press('enter')  # Confirm the data entry

def select_range(start_cell, end_cell):
    """
    Selects a range of cells in Excel.

    Args:
        start_cell (str): The address of the starting cell (e.g., 'A1', 'B3').
        end_cell (str): The address of the ending cell (e.g., 'C5', 'D10').
    """
    move_to_cell(start_cell)  # Move to the starting cell
    pyautogui.hotkey('shift', 'ctrl', 'g')  # Open the "Go To" dialog for range selection
    pyautogui.write(end_cell)  # Enter the ending cell address
    pyautogui.press('enter')  # Confirm the range selection


In [2]:
select_range.__doc__

"\n    Selects a range of cells in Excel.\n\n    Args:\n        start_cell (str): The address of the starting cell (e.g., 'A1', 'B3').\n        end_cell (str): The address of the ending cell (e.g., 'C5', 'D10').\n    "